# Overview

Produce a quarterly KPI summary that compares actual TPCH sales against your targets, computes year-over-year growth, and flags performance status by nation. The end results of this should be a **VIEW**.

Imagine you need to deliver a “Quarterly Performance Dashboard” for regional managers, showing for each nation:
1.  **Actual revenue** vs. **target revenue**
2.  **Order count** vs. **target order count**
3.  **% of target achieved**
4.  **Year-over-year (YoY) revenue growth**
5.  A simple **status flag** (“On Track”, “At Risk”, “Behind”)

> **Note:** Upload the **target** files to a volume, these target files should then be read into a CTE using SQL so they can be joined with the rest of your TPCH orders data.

> **Note:** Don't forget to define the `target catalog`, `target schema`, and `target view` name using the widgets to define where your final view will live. You may need to run cell 2 in order for the widgets to appear at the top of the notebook.

> **Note:** In the code below you wil see the `IDENTIFIER` clause used when creating the view. The clause enables SQL injection safe parameterization of SQL statements. [You can read more here.](https://learn.microsoft.com/en-us/azure/databricks/sql/language-manual/sql-ref-names-identifier-clause). The `IDENTIFIER` clause is used when using variables (or parameters) and interprets a string as a:
>>  - table or view name
>>  - function name
>>  - column name
>>  - field name
>>  - schema name
>>  - catalog name

In [0]:
/*************************************************************************
STEP 1: Create variables to define the name of the view you want to create
**************************************************************************/
DECLARE OR REPLACE VARIABLE target_catalog STRING DEFAULT 'ae_prod';
DECLARE OR REPLACE VARIABLE target_schema STRING DEFAULT 'tpch';
DECLARE OR REPLACE VARIABLE target_view STRING DEFAULT 'tpch_order_view';
-- create a variable for the fully qualified table name
DECLARE OR REPLACE VARIABLE target_fq_view STRING DEFAULT 'samples.samples.tpch.order_view';

-- set variables
SET VAR target_catalog = 'ae_prod';
SET VAR target_schema = :target_schema;
SET VAR target_view = :target_view;
-- concatenate the variables to create the fully qualified table name
SET VAR target_fq_view = target_catalog || '.' || target_schema || '.' || target_view;

--show the variable values
SELECT target_catalog, target_schema, target_view, target_fq_view

In [0]:
/*************************************
STEP 2: Add syntax to create the view
************************************/

CREATE OR REPLACE VIEW IDENTIFIER(target_fq_view)
  AS (

/*******************************
STEP 3: Load your CSV of targets
********************************/
WITH raw_targets AS (
  SELECT *
  FROM read_files('/Volumes/ddavis_dev/demo/volume_for_files/targets/',
    format => 'csv',
    header => true,
    inferSchema => true
  )
),

/********************************************
STEP 4: Build actuals by quarter & nation
********************************************/
actuals_qtr AS (
  SELECT
    concat(year(o.o_orderdate), '-Q', ceil(month(o.o_orderdate)/3.0)) AS year_quarter,
    n.n_name                                               AS nation,
    SUM(l.l_extendedprice * (1 - l.l_discount))           AS actual_revenue,
    COUNT(DISTINCT o.o_orderkey)                           AS actual_order_count
  FROM samples.tpch.orders AS o
  JOIN samples.tpch.lineitem AS l
    ON o.o_orderkey = l.l_orderkey
  JOIN samples.tpch.customer AS c
    ON o.o_custkey   = c.c_custkey
  JOIN samples.tpch.nation AS n
    ON c.c_nationkey = n.n_nationkey
  GROUP BY 1, 2
),

/*******************************************************
STEP 5: Join actuals to targets & compute % of target
*******************************************************/
kpi_joined AS (
  SELECT
    t.year,
    t.quarter,
    CONCAT(t.year, t.quarter) as year_quarter,
    t.nation,
    a.actual_revenue,
    t.target_revenue,
    ROUND(a.actual_revenue / t.target_revenue * 100, 1) AS pct_of_revenue_target,
    a.actual_order_count,
    t.target_order_count
  FROM raw_targets AS t
  LEFT JOIN actuals_qtr AS a
    ON CONCAT(t.year, '-', t.quarter) = a.year_quarter
   AND t.nation       = a.nation
),

/*******************************************************
STEP 6: Compute YoY growth and performance flag
*******************************************************/
kpi_yoy AS (
  SELECT
    k.*,
    -- extract numeric year and quarter
    CAST(split(k.year_quarter,'Q')[0] AS INT) AS yr,
    CAST(split(k.year_quarter,'Q')[1] AS INT) AS qtr
  FROM kpi_joined AS k
),

-- now join kpi_yoy to itself to get last-year’s revenue
kpi_with_growth AS (
  SELECT
    curr.year_quarter,
    curr.nation,
    curr.actual_revenue,
    curr.target_revenue,
    curr.pct_of_revenue_target,
    curr.actual_order_count,
    curr.target_order_count,
    prev.actual_revenue AS actual_revenue_prev,
    ROUND(
      (curr.actual_revenue - prev.actual_revenue) / prev.actual_revenue * 100, 1
    ) AS yoy_revenue_pct_change,
    CASE
      WHEN curr.pct_of_revenue_target >= 100 THEN 'On Track'
      WHEN curr.pct_of_revenue_target >= 90  THEN 'At Risk'
      ELSE 'Behind'
    END AS performance_status
  FROM kpi_yoy AS curr
  LEFT JOIN kpi_yoy AS prev
    ON curr.nation = prev.nation
   AND curr.yr    = cast(prev.yr as int) + 1
   AND curr.qtr   = prev.qtr
)


/*************************************
STEP 7: Final view definition
*************************************/
SELECT
  year_quarter,
  actual_revenue,
  target_revenue,
  pct_of_revenue_target,
  actual_order_count,
  target_order_count,
  actual_revenue_prev,
  yoy_revenue_pct_change,
  performance_status
FROM kpi_with_growth
ORDER BY year_quarter, nation
);